#### simple cnn with cifar10

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

(train_imgs, train_labels), (test_imgs, test_labels) = datasets.cifar10.load_data()
train_imgs, test_imgs = train_imgs / 255.0, test_imgs / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(train_imgs, train_labels, epochs=10, validation_data=(test_imgs, test_labels))

#### LeNet5

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(filters=6, kernel_size=5, strides=1, activation="tanh", input_shape=(28, 28, 1), padding="same"))
model.add(AveragePooling2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(filters=16, kernel_size=5, strides=1, activation="tanh", input_shape=(28, 28, 1), padding="valid"))
model.add(AveragePooling2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(filters=120, kernel_size=5, strides=1, activation="tanh", input_shape=(28, 28, 1), padding="valid"))
model.add(Flatten())
model.add(Dense(units=84, activation="tanh"))
model.add(Dense(units=10, activation="softmax"))
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

#### simple gan for mnist

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

(train_images, train_labels), _ = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(256)

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((7, 7, 256)))
    
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1,1), padding="same", use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2,2), padding="same", use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2,2), padding="same", use_bias=False, activation="tanh"))
    
    return model

generator = make_generator_model()

def make_discriminator_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding="same", input_shape=[28,28,1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(1))
    
    return model

discriminator = make_discriminator_model()

cross_entropy = tf.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optim = tf.keras.optimizers.Adam(1e-4)
discriminator_optim = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([256, 100])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optim.apply_gradients(zip(gen_grads, generator.trainable_variables))
    discriminator_optim.apply_gradients(zip(disc_grads, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        print("\nat epoch " + str(epoch))
        for image_batch in dataset:
            train_step(image_batch)
        test_display = generator(tf.random.normal([2,100]), training=False)
        plt.imsave(str(epoch) + ".jpg", test_display[0,:,:,0] * 127.5 + 127.5, cmap='gray')

#### simple style transfer

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import IPython.display as display

content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('gogh.jpg','https://cdn.britannica.com/78/43678-050-F4DC8D93/Starry-Night-canvas-Vincent-van-Gogh-New-1889.jpg')

def load_img(path):
    maxdim = 512
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    scale = maxdim / max(shape)
    
    img = tf.image.resize(img, tf.cast(shape * scale, tf.int32))
    img = tf.expand_dims(img, 0)
    return img

content = load_img(content_path)
style = load_img(style_path)

def imshow(img):
    if len(img.shape) > 3:
        img = tf.squeeze(img)
    plt.imshow(img)

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

content_layers = [
    'block5_conv2'
]

style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1',
]

def vgg_layers(layers):
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(layer).output for layer in layers]
    model = tf.keras.Model([vgg.input], outputs)
    return model

def gram_matrix(inp):
    result = tf.linalg.einsum('bijc,bijd->bcd', inp, inp)
    inpshape = tf.shape(inp)
    num_elems = tf.cast(inpshape[1] * inpshape[2], tf.float32)
    return result / num_elems

class StyleContentModel(tf.keras.models.Model):
    def __init__(self):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.vgg.trainable = False
    
    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed)
        
        style_outputs, content_outputs = (outputs[:len(self.style_layers)], outputs[len(self.style_layers):])
        style_outputs = [gram_matrix(output) for output in style_outputs]
        
        content_dict = {
            content_name : value
            for content_name, value in zip(self.content_layers, content_outputs)
        }
        
        style_dict = {
            style_name : value
            for style_name, value in zip(self.style_layers, style_outputs)
        }
        
        return {'content' : content_dict, 'style' : style_dict}

extractor = StyleContentModel()

style_targets = extractor(style)['style']
content_targets = extractor(content)['content']

def clipp(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

style_weight = 1e-2
content_weight = 1e4

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name]) ** 2) for name in style_outputs.keys()])
    style_loss *= style_weight / len(style_layers)
    
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name]) ** 2) for name in content_outputs.keys()])
    content_loss *= content_weight / len(content_layers)
    
    return style_loss + content_loss

@tf.function
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)
    
    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clipp(image))

image = tf.Variable(content)
display.display(tensor_to_image(image))

epochs = 20
steps_per_epoch = 100

for i in range(epochs):
    for j in range(steps_per_epoch):
        train_step(image)
    display.clear_output(wait=True)
    display.display(tensor_to_image(image))

#### todo